<a href="https://colab.research.google.com/github/thofes/SJ_distance/blob/main/Label_Skijumper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Connect GDrive
from google.colab import drive
drive.mount('/content.gdrive')
"""
#init Repository - YOLOv5
%cd /content


!git clone https://github.com/ultralytics/yolov5 # clone
%cd yolov5
%pip install -qr requirements.txt  # install

#import detect.py and dataloaders.py from github
"""

Mounted at /content.gdrive


'\n#init Repository - YOLOv5\n%cd /content\n\n\n!git clone https://github.com/ultralytics/yolov5 # clone\n%cd yolov5\n%pip install -qr requirements.txt  # install\n\n#import detect.py and dataloaders.py from github\n'

In [2]:
from numpy.core.fromnumeric import shape


import os
from os.path import exists, join, basename, splitext
import math
from collections import OrderedDict
import json
import numpy as np
import cv2

save_counter = 1

directory_label = 'labels'
#parent_dir = Path_Pic + '/exp/'
parent_dir = '/content.gdrive/MyDrive/training_data'
path_label = os.path.join(parent_dir, directory_label)
os.mkdir(path_label)
directory_data = 'data'
path_data = os.path.join(parent_dir, directory_data)
os.mkdir(path_data)
#directory_bb = 'bb'
#path_bb = os.path.join(parent_dir, directory_bb)
#os.mkdir(path_bb)



PATH = "/content.gdrive/MyDrive/Manually_Labeled_01042022"
label= []
try:
    label = next(os.walk(PATH))[1]
except StopIteration:
    pass
#data["labels"].append(label)
print(label)

savecounter = 0

for l in label:
  Path_jumpers = PATH + '/' + l
  print(Path_jumpers)
  Jumpers = []
  try:
    Jumpers = next(os.walk(Path_jumpers))[1]
  except StopIteration:
    pass
  print("Anzahl: " , len(Jumpers))
  

  for j in Jumpers:
    Path_Pic = Path_jumpers + '/' + j
    print(Path_Pic)
    
#if exp vorhanden or not
      
    Path_check = Path_Pic + '/exp'


    if os.path.isdir(Path_check):




        #%cd /content/yolov5
        #!python detect.py --save-crop --source '{Path_Pic}' --project '{Path_Pic}' --save-txt


        with open(Path_Pic + '/exp/predictions.json', "r") as fp:
            data = json.load(fp)

        # convert lists to numpy arrays
        x_center = np.array(data["x_center"])
        y_center = np.array(data["y_center"])
        x1 = np.array(data["x1"])
        x2 = np.array(data["x2"])
        y1 = np.array(data["y1"])
        y2 = np.array(data["y2"])
        conf = np.array(data["conf"])
        det = np.array(data["det"])



        path = Path_Pic + '/1.jpg'
        #print(path)
        im = cv2.imread(path)
        height, width, channels = im.shape
        #width = 1920
        #height = 1080

        x_crop = []
        y_crop = []



        #detect movement mit check vor und zurück


        athlet_det_index = []


        for y in range(21):
          #print("loop" , y)
          if det[y] != 0: #wird eventuellenötigt, falls keine detections gemacht wurden
            #print("detvorhanden")
            distance_next = []
            distance_index_next = []
            distance_previous = []
            distance_index_previous = []
            valid_dist_index = []
            y_center_abs_per_det = []

        # calculate distances and save relevant information



            if y == 0: #beim ersten Bild wird der Abstand der BB zum Folgebild berrechnet

                for x in range(det[y+1]):            
                    for i in range(det[y]):
                        distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_next.append(i)

                

            elif y > 0 and y < 20: #bei den mittleren Bildern wird der Abstand der BB zum Vorbild und Folgebild berrechnet
                
                for x in range(det[y+1]):
                    for i in range(det[y]):
                        distance_next.append(math.sqrt((x_center[sum(det[:y+1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y+1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_next.append(i)

                for x in range(det[y-1]):
                    for i in range(det[y]):
                        distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_previous.append(i)

                
                
            else: # Beim letzten Bild wird der Abstand zur BB des vorletzten Bildes berechnet
                for x in range(det[y-1]):
                    
                    for i in range(det[y]):
                        distance_previous.append(math.sqrt((x_center[sum(det[:y-1])+x]-x_center[sum(det[:y])+i])**2 + (y_center[sum(det[:y-1])+x]-y_center[sum(det[:y])+i])**2))
                        distance_index_previous.append(i)


            #print("distance_next: ",distance_next)
            #print("distance_previous:",distance_previous)


        # Movement detector - speichert alle Detections, die der Athlet sein könnte

            if y == 0: # beim ersten Bild nur vergleich mit dem Nächsten und Einschränkung durch eine Vorgabe der Distanz
                #print("first BB")
                for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild    
                    if 20 < x < 75 and abs(y_center[i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                        valid_dist_index.append(distance_index_next[i])


            elif athlet_det_index[y-1] == 999: #and len(distance_index_next) != 0: # falls beim letzten Bild keine Detection gemacht wurde 
                #print("last det 999 BP")
                last_det_index = 999
                counter = 0
                for index in athlet_det_index: # letzte gültige detection finden
                    if index != 999:
                        last_det_index = index
                        counter = 0
                    else:
                        counter = counter + 1

                if last_det_index != 999: # falls bereits valide detection gefunden wurde
                    # prüfung zum nächsten Bild
                    for i, x in enumerate(distance_next):     
                        if 20 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                            valid_dist_index.append(distance_index_next[i])
                    # y Abstand zur letzten detection
                    for i in range(det[y]):
                        if abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1-counter])+last_det_index]) < 30:
                            valid_dist_index.append(i)
                

                else: # wenn keine vorhanden ist nur eine Einschränkung wie beim ersten Bild
                #eventuell besser nur y Koordinate?
                    for i, x in enumerate(distance_next):     
                        if 20 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y+1])+i//det[y]]) < 30:
                            valid_dist_index.append(distance_index_next[i])

            elif y > 0 and y < 20: # bei den mittleren Bildern werden beide Richtungen eingeschränkt  und zusätzlich wird auch geprüft, ob die Y Koordinaten nicht zu weit abweichen
                #print("middel BB")
                for i, x in enumerate(distance_next): # Einschränkung der Detections anhand des Abstandes zum nächsten Bild und y Abstand zur letzten detection
                    #print("ynext: ", abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                    if 20 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_next[i]) 
          

                for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild
                    #print("yprevious: ", abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                    if 20 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_previous[i])
                

            elif y == 20: # beim letzten Bid wieder nur vergleich mit dem vorherigen Bild
                #print("last BB")
                for i, x in enumerate(distance_previous): # Einschränkung der Detections anhand des Abstandes zum vorherigen Bild sowie Validierung der Y Koordinate
                    if 20 < x < 75 and abs(y_center[sum(det[:y])+i%det[y]]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]) < 30:
                        valid_dist_index.append(distance_index_previous[i])




        # in der Liste valid_dist_index sind die Indexe der validen Detections im Bild gespeichert (Validierung mit Movement detector)
            #print("validdetList: ", valid_dist_index)

        # Anpassungen der Liste der validen Detections - finden des Athleten

            if len(valid_dist_index) > 1: # Falls für eine Bildbox mehrere mögliche Verbindungen zum Folgebild gefunden wurden, werden die Duplikate entfernt
                valid_dist_index = list(OrderedDict.fromkeys(valid_dist_index))

            if len(valid_dist_index) == 1: # wenn nur eine vorhanden ist wird diese verwendet
                athlet_det_index.append(valid_dist_index[0])

            elif len(valid_dist_index) == 0: # falls keine validen aus dem Movement detector vorhanden sind wird 999 (keine detection) ergänzt 
                athlet_det_index.append(999)

            elif y == 0: # Für das erste Bild mit mehreren validen detections
                
                for dist in valid_dist_index: # zentralste detection auf Y Achse
                    y_center_abs_per_det.append(abs(y_center[dist]-height/2))
                closest = min(y_center_abs_per_det)
                min_index = y_center_abs_per_det.index(closest)            
                athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert



            elif y < 21: # Für alle weiteren Bilder mit mehreren validen detections
                dist_to_previous_y_center = [] # Kleinster y Abstand zur letzten detections wird gesucht

                if athlet_det_index[y-1] == 999: # letzes Bild keine detection
                    last_det_index = 999
                    counter = 0
                    for index in athlet_det_index: # letzte gültige detection finden
                        if index != 999:
                            last_det_index = index
                            counter = 0
                        else:
                            counter = counter + 1            
                    
                    if last_det_index != 999: # falls eine gültige detection gefunden wurde
                        for i, x in enumerate(valid_dist_index): # Abstand der Y Kooridnaten zur letzten Detection
                            dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1-counter])+last_det_index]))
                        closest = min(dist_to_previous_y_center)
                        min_index = dist_to_previous_y_center.index(closest)
                        athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert

                    else: # falls keine gültige detection gefunden wurde - wie bei Bild 1
                        for dist in valid_dist_index:
                            y_center_abs_per_det.append(abs(y_center[dist]-height/2))                          
                        closest = min(y_center_abs_per_det)
                        min_index = y_center_abs_per_det.index(closest)            
                        athlet_det_index.append(valid_dist_index[min_index]) # index der Detection vom ersten Bild gespeichert

                else:
                    for i, x in enumerate(valid_dist_index):
                        dist_to_previous_y_center.append(abs(y_center[sum(det[:y])+x]-y_center[sum(det[:y-1])+athlet_det_index[y-1]]))
                    closest = min(dist_to_previous_y_center)
                    min_index = dist_to_previous_y_center.index(closest)
                    athlet_det_index.append(valid_dist_index[min_index]) # index der Detection wird gespeichert 





          else:
              athlet_det_index.append(999) # falls keine detections im Bild vorhanden sind
          #print("athlet_det_index: ", athlet_det_index)
        print("athlet_det_index: ", athlet_det_index)



        #bounding box
        for i,x in enumerate(athlet_det_index):

            if x != 999: # zuschneiden laut Koordinaten der BB (50 Pixel vergrößert)
                x_1 = x1[sum(det[:i])+x]
                x_2 = x2[sum(det[:i])+x]
                y_1 = y1[sum(det[:i])+x]
                y_2 = y2[sum(det[:i])+x]


                line = (0, x_center[sum(det[:i])+x]/width, y_center[sum(det[:i])+x]/height, (x_2-x_1)/width, (y_2-y_1)/height) 
                with open(path_label + '/' + str(save_counter)+ '.txt', 'w') as f:
                    f.write(('%g ' * len(line)).rstrip() % line )


                path = Path_Pic + '/' + str(i+1) + '.jpg'
                #print(path)
                im = cv2.imread(path)
                save_path_original = path_data +'/' + str(save_counter) + '.jpg'
                cv2.imwrite(save_path_original, im)
                #print(x_1, x_2, y_1, y_2)
                #crop = im[int(y_1):int(y_2), int(x_1):int(x_2)] 
                #save_dir.parent.mkdir(parents=True, exist_ok=True)  # make directory
                
                # represents the top left corner of rectangle
                start_point = (int(x_1), int(y_1))
              
                # Ending coordinate, here (220, 220)
                # represents the bottom right corner of rectangle
                end_point = (int(x_2), int(y_2))
              
                # Blue color in BGR
                color = (255, 0, 0)
              
                # Line thickness of 2 px
                thickness = 2

                #print("start", start_point)
                #print("end", end_point)
                #image_with_box = cv2.rectangle(im, start_point, end_point, color, thickness)

                #save_path = path_bb +'/' + str(save_counter) + 'bb.jpg'
                
                #cv2.imwrite(save_path, image_with_box) # 
                

                save_counter += 1




['Telemark', 'NoTelemark']
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark
Anzahl:  111
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/913
athlet_det_index:  [2, 3, 3, 4, 3, 3, 999, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/84
athlet_det_index:  [999, 999, 0, 0, 999, 999, 0, 0, 999, 999, 0, 0, 0, 999, 999, 0, 0, 0, 999, 999, 999]
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/747
athlet_det_index:  [1, 2, 0, 2, 1, 2, 3, 1, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3]
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/901
athlet_det_index:  [2, 2, 999, 2, 1, 2, 0, 1, 999, 0, 999, 1, 0, 999, 999, 999, 999, 0, 0, 0, 1]
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/845
athlet_det_index:  [999, 0, 2, 3, 3, 3, 3, 3, 999, 3, 3, 999, 3, 3, 3, 3, 3, 3, 3, 2, 3]
/content.gdrive/MyDrive/Manually_Labeled_01042022/Telemark/914
athlet_det_index:  [999, 999, 999, 5, 5, 2, 5, 5, 5, 5, 5, 5, 5, 